# Tool UseとRAG

LLM セクションの学習ステップ 6/8。
Tool UseとRAGでは、モデル呼び出し以前に入力設計と評価設計を固める習慣を作ります。

このステップの到達目標: プロンプト、事前学習、微調整、RAG、効率化を実装視点で横断し、設計判断を言語化できる状態にします。
前提: 確率的な予測モデルの見方と、深層学習の基礎が前提です。

今回の中心語: 「トークン」、「事前学習」、「微調整」、「RAG」、「推論最適化」、「Tool」、「UseとRAG」
前ステップ「ハルシネーションとRLHF」では ハルシネーション観測の基礎 → プロンプトを構造化する を確認しました。
ここまでに登場した語: 「トークン」、「事前学習」、「微調整」、「RAG」、「推論最適化」、「プロンプトエンジニアリング」、「スケーリング則」、「ファインチューニング」、「ハルシネーションとRLHF」
セクション全体のゴール: プロンプト、事前学習、微調整、RAG、効率化を実装視点で横断し、設計判断を言語化できる状態にします。

## 観察 1: トークン近似を体験する

最初に、入力長とトークン量の関係を簡易計測します。コスト管理の第一歩です。

In [ ]:
text = '大規模言語モデルは文脈の与え方で応答品質が大きく変わる。'
char_len = len(text)
space_tokens = text.split()
rough_tokens = max(1, char_len // 2)
print('chars=', char_len, 'space_tokens=', len(space_tokens), 'rough_tokens=', rough_tokens)

厳密なトークン化ではありませんが、入力を短く保つ設計感覚を作るには十分です。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 2: プロンプトを構造化する

次に、指示・制約・出力形式を分離したテンプレートを作ります。曖昧さを減らすための実装技法です。

In [ ]:
instruction = '勾配降下法を初学者向けに説明する'
constraints = ['120字以内', '比喩は1つまで', '最後に要点を1行でまとめる']
prompt = f"指示: {instruction}\n制約: {'; '.join(constraints)}\n出力:"
print(prompt)
print('prompt_chars=', len(prompt))

この形にすると、失敗原因を特定しやすくなります。品質改善は原因分離のしやすさで決まります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 計算の対応表

1. $p_theta(x_t | x_<t)$
2. $L_CE = - sum_t log p_theta(x_t | x_<t)$

## 観察 3: RAG入力を作る

検索結果をそのまま貼るのではなく、質問との関係を明示して結合します。

In [ ]:
question = 'ベルマン最適方程式を1文で説明して'
chunks = ['最適価値は将来報酬期待値の最大化で定義される', '再帰構造により逐次更新できる']
ranked = sorted(chunks, key=lambda c: question.count('価値') + c.count('価値'), reverse=True)
context = '\n'.join(f'- {c}' for c in ranked)
print('context_for_answer=\n' + context)

この前処理を入れるだけでも、回答の一貫性と根拠明示が改善します。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 4: 評価項目を数値化する

次に、回答を点検するための簡易スコアを定義します。評価軸を言語化すると改善が継続できます。

In [ ]:
answer = 'ベルマン方程式は、今の価値を次の価値で更新する再帰式です。'
checks = {'length_ok': len(answer) <= 120, 'has_keyword': '価値' in answer, 'has_recurrence': '再帰' in answer}
score = sum(1 for v in checks.values() if v) / len(checks)
print('checks=', checks)
print('score=', round(score, 3))

このような軽量評価でも、改善方向を揃える効果があります。実務ではこの評価軸をチームで共有します。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 観察 5: 推論コストを見積もる

最後に、入力と出力の長さから概算コストを計算します。モデル選択は性能だけでなく費用とのバランスが必要です。

In [ ]:
input_tokens = 320
output_tokens = 180
price_per_1k = 0.0012
cost = (input_tokens + output_tokens) / 1000 * price_per_1k
print('estimated_cost=', round(cost, 6))

この見積もりを運用前に作ると、スケール時の予算超過を防ぎやすくなります。

この節では、トークン が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 要点整理

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. プロンプトだけで全問題を解決しようとする
2. 評価指標を決めずに改善を繰り返す
3. コストと品質のバランスを見ない

次は学習ステップ 7/8「ドメイン特化」へ進み、今回のコードとの差分を確認してください。